In [26]:
#0.Import libraries
import os
import time
import requests
import math
import json
from dotenv import load_dotenv
from eth_account import Account
from eth_abi import encode
from web3 import Web3, EthereumTesterProvider
from eth_account.messages import encode_defunct

#1.Load Keys & Set Base Endpoint
load_dotenv('keys.env')
API_KEY = os.getenv("API_KEY")
API_SECRET = os.getenv("API_SECRET").encode()  # bytesにする必要あり
ASTER_USER = os.getenv("ASTER_USER")
ASTER_SIGNER = os.getenv("ASTER_SIGNER")
ASTER_PK = os.getenv("ASTER_PRIVATE_KEY")
BASE = "https://fapi.asterdex.com"

#2.Test
nonce = math.trunc(time.time()*1000000)
my_dict = {}

def _trim_dict(my_dict) :
    for key in my_dict:
        value = my_dict[key]
        if isinstance(value, list):
            new_value = []
            for item in value:
                if isinstance(item, dict):
                    new_value.append(json.dumps(_trim_dict(item)))
                else:
                    new_value.append(str(item))
            my_dict[key] = json.dumps(new_value)
            continue
        if isinstance(value, dict):
            my_dict[key] = json.dumps(_trim_dict(value))
            continue
        my_dict[key] = str(value)
    return my_dict

my_dict = {key: value for key, value in my_dict.items() if  value is not None}
my_dict['recvWindow'] = 50000
my_dict['timestamp'] = int(round(time.time()*1000))

_trim_dict(my_dict)
json_str = json.dumps(my_dict, sort_keys=True).replace(' ', '').replace('\'','\"')
print(json_str)

encoded = encode(['string', 'address', 'address', 'uint256'], [json_str, ASTER_USER, ASTER_SIGNER, nonce])
print(encoded.hex())

keccak_hex =Web3.keccak(encoded).hex()
print(keccak_hex)

signable_msg = encode_defunct(hexstr=keccak_hex)
signed_message = Account.sign_message(signable_message=signable_msg, private_key=ASTER_PK)
signature =  '0x'+signed_message.signature.hex()
print(signature)

my_dict['nonce'] = nonce
my_dict['user'] = ASTER_USER
my_dict['signer'] = ASTER_SIGNER
my_dict['signature'] = '0x'+signed_message.signature.hex()
url ='https://fapi.asterdex.com/fapi/v3/positionRisk'
headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'User-Agent': 'PythonApp/1.0'
}
res = requests.get(url,params=my_dict,headers=headers)
print(url)

{"recvWindow":"50000","timestamp":"1758662320104"}
0000000000000000000000000000000000000000000000000000000000000080000000000000000000000000ca44395071da0d5d8177396dfd1de6deb4cb3e71000000000000000000000000ca44395071da0d5d8177396dfd1de6deb4cb3e7100000000000000000000000000000000000000000000000000063f7e7adea35900000000000000000000000000000000000000000000000000000000000000327b227265637657696e646f77223a223530303030222c2274696d657374616d70223a2231373538363632333230313034227d0000000000000000000000000000
0x02b1f98f01c0a97d70cb5b521663ebdb35451de48a13e4b50014c741c5aaf162
0x0x5d1001424f163886079083d2399298b739e13f8463cc19d4938de8faa58a7ae94bd0f62dd6f2444c8663f50b878dd2a73c7792933158a7564a41907f624de3751b
https://fapi.asterdex.com/fapi/v3/positionRisk


In [31]:
#Python 3.9.6
#Python 3.9.6
#eth-account~=0.13.7
#eth-abi~=5.2.0
#web3~=7.11.0
#requests~=2.32.3

import json
import math
import time
import requests

from eth_abi import encode
from eth_account import Account
from eth_account.messages import encode_defunct
from web3 import Web3

load_dotenv('keys.env')

user = os.getenv("ASTER_USER")
signer = os.getenv("ASTER_SIGNER")
priKey = os.getenv("ASTER_PRIVATE_KEY")
host = 'https://fapi.asterdex.com'
placeOrder = {'url': '/fapi/v3/order', 'method': 'POST',
              'params':{'symbol': 'SANDUSDT', 'positionSide': 'BOTH', 'type': 'LIMIT', 'side': 'BUY',
	         'timeInForce': 'GTC', 'quantity': "30", 'price': 0.325,'reduceOnly': True}}
getOrder = {'url':'/fapi/v3/order','method':'GET','params':{'symbol':'SANDUSDT','side':"BUY","type":'LIMIT','orderId':2194215}}
getPositionInfo = {'url':'/fapi/v3/positionRisk', 'method':'GET', 'params':{}}

def call(api):
    nonce = math.trunc(time.time() * 1000000)
    my_dict = api['params']
    send(api['url'],api['method'],sign(my_dict,nonce))

def sign(my_dict,nonce):
    my_dict = {key: value for key, value in my_dict.items() if  value is not None}
    my_dict['recvWindow'] = 50000
    my_dict['timestamp'] = int(round(time.time()*1000))
    msg = trim_param(my_dict,nonce)
    signable_msg = encode_defunct(hexstr=msg)
    signed_message = Account.sign_message(signable_message=signable_msg, private_key=priKey)
    my_dict['nonce'] = nonce
    my_dict['user'] = user
    my_dict['signer'] = signer
    my_dict['signature'] = signed_message.signature.hex()

    print(my_dict['signature'])
    return  my_dict

def trim_param(my_dict,nonce) -> str:
    _trim_dict(my_dict)
    json_str = json.dumps(my_dict, sort_keys=True).replace(' ', '').replace('\'','\"')
    print(json_str)
    encoded = encode(['string', 'address', 'address', 'uint256'], [json_str, user, signer, nonce])
    print(encoded.hex())
    keccak_hex =Web3.keccak(encoded).hex()
    print(keccak_hex)
    return keccak_hex

def _trim_dict(my_dict) :
    for key in my_dict:
        value = my_dict[key]
        if isinstance(value, list):
            new_value = []
            for item in value:
                if isinstance(item, dict):
                    new_value.append(json.dumps(_trim_dict(item)))
                else:
                    new_value.append(str(item))
            my_dict[key] = json.dumps(new_value)
            continue
        if isinstance(value, dict):
            my_dict[key] = json.dumps(_trim_dict(value))
            continue
        my_dict[key] = str(value)

    return my_dict

def send(url, method, my_dict):
    url = host + url
    print(url)
    print(my_dict)
    if method == 'POST':
        headers = {
            'Content-Type': 'application/x-www-form-urlencoded',
            'User-Agent': 'PythonApp/1.0'
        }
        res = requests.post(url, data=my_dict, headers=headers)
        print(res.text)
    if method == 'GET':
        res = requests.get(url, params=my_dict)
        print(res.text)
    if method == 'DELETE':
        res = requests.delete(url, data=my_dict)
        print(res.text)

call(getPositionInfo)


{"recvWindow":"50000","timestamp":"1758663092233"}
0000000000000000000000000000000000000000000000000000000000000080000000000000000000000000ca44395071da0d5d8177396dfd1de6deb4cb3e71000000000000000000000000ca44395071da0d5d8177396dfd1de6deb4cb3e7100000000000000000000000000000000000000000000000000063f7ea8e4638b00000000000000000000000000000000000000000000000000000000000000327b227265637657696e646f77223a223530303030222c2274696d657374616d70223a2231373538363633303932323333227d0000000000000000000000000000
0x6d32111e63fb04362f777452c83b91c24c144fd0a2d0639643816891ddaedd09
0x784489768449c93cedf69da29e0ab63e2f5d6354897345ac43dfef6fd44a2efb3bdeba3a376eac14e006a3617c2e9418705076d14e57b05bbe701e2529bd1e0d1b
https://fapi.asterdex.com/fapi/v3/positionRisk
{'recvWindow': '50000', 'timestamp': '1758663092233', 'nonce': 1758663092233099, 'user': '0xCA44395071da0D5D8177396Dfd1dE6DEB4cB3E71', 'signer': '0xCA44395071da0D5D8177396Dfd1dE6DEB4cB3E71', 'signature': '0x784489768449c93cedf69da29e0ab63e2f5d6354897345

In [32]:
import os
import json
import math
import time
import requests

from dotenv import load_dotenv
from eth_abi import encode
from eth_account import Account
from eth_account.messages import encode_defunct
from web3 import Web3

load_dotenv('keys.env')

user   = os.getenv("ASTER_USER")
signer = os.getenv("ASTER_SIGNER")
priKey = os.getenv("ASTER_PRIVATE_KEY")
host   = 'https://fapi.asterdex.com'

getPositionInfo = {
    'url': '/fapi/v3/positionRisk',
    'method': 'GET',
    'params': {
        # 'symbol': 'SANDUSDT',   # 必要なら有効化
    }
}

def call(api):
    nonce = math.trunc(time.time() * 1_000_000)  # μs
    params = dict(api['params'])                 # defensive copy
    signed = sign(params, nonce)
    send(api['url'], api['method'], signed)

def sign(my_dict, nonce):
    # None除去
    my_dict = {k: v for k, v in my_dict.items() if v is not None}
    my_dict['recvWindow'] = 50000
    my_dict['timestamp']  = int(round(time.time()*1000))

    # 署名メッセージ生成（nonce は署名対象に含めない仕様）
    msg_hex = trim_param(my_dict, nonce)
    signable_msg = encode_defunct(hexstr=msg_hex)
    signed = Account.sign_message(signable_msg, private_key=priKey)

    # ここが重要：追加する4項目は**必ず文字列**化
    my_dict['nonce']     = str(nonce)
    my_dict['user']      = str(user)
    my_dict['signer']    = str(signer)
    sig_hex = signed.signature.hex()
    my_dict['signature'] = sig_hex if sig_hex.startswith('0x') else '0x' + sig_hex

    return my_dict

def trim_param(my_dict, nonce) -> str:
    _trim_dict(my_dict)  # 既存キーを全部 str 化
    json_str = json.dumps(my_dict, sort_keys=True, separators=(',', ':'))
    encoded = encode(['string', 'address', 'address', 'uint256'], [json_str, user, signer, nonce])
    return Web3.keccak(encoded).hex()

def _trim_dict(my_dict):
    for k in list(my_dict.keys()):
        v = my_dict[k]
        if isinstance(v, list):
            new_v = []
            for item in v:
                new_v.append(json.dumps(_trim_dict(item)) if isinstance(item, dict) else str(item))
            my_dict[k] = json.dumps(new_v)
        elif isinstance(v, dict):
            my_dict[k] = json.dumps(_trim_dict(v))
        else:
            my_dict[k] = str(v)
    return my_dict

def send(url, method, params):
    url = host + url
    headers = {'User-Agent': 'PythonApp/1.0'}
    if method == 'POST':
        headers['Content-Type'] = 'application/x-www-form-urlencoded'
        res = requests.post(url, data=params, headers=headers, timeout=30)
    elif method == 'GET':
        res = requests.get(url, params=params, headers=headers, timeout=30)
    elif method == 'DELETE':
        res = requests.delete(url, data=params, headers=headers, timeout=30)
    else:
        raise ValueError(f"Unsupported method: {method}")

    print(url)
    print(params)
    print(res.status_code, res.text)
    return res

# 実行
call(getPositionInfo)


https://fapi.asterdex.com/fapi/v3/positionRisk
{'recvWindow': '50000', 'timestamp': '1758663238147', 'nonce': '1758663238147290', 'user': '0xCA44395071da0D5D8177396Dfd1dE6DEB4cB3E71', 'signer': '0xCA44395071da0D5D8177396Dfd1dE6DEB4cB3E71', 'signature': '0xe732d7725be8ac71db91ae6a149cdca5d974b549ba594da12780ea2c2d33376b709fc3150b9b5293eba95257481137f0af248a6b99a7b8bd52110a1dd994a3441b'}
500 <!DOCTYPE html>
<html>
  <head>
    <style>
      * {
        box-sizing: border-box;
      }
      html,
      body {
        margin: 0;
        border: 0;
      }
      .p-8 {
        padding: 8px;
      }
      .bg-0b0d12 {
        background-color: #0b0d12;
      }
      .minH-100p {
        min-height: 100%;
      }
      .absolute {
        position: absolute;
      }
      .w-100p {
        width: 100%;
      }
      .bottom-0 {
        bottom: 0px;
      }
      .top-0 {
        top: 0px;
      }
      .column-toCenterX {
        align-items: center;
      }
      .flexDirection-column {
    

In [33]:
# Python 3.9.6
# pip install eth-account==0.13.7 eth-abi==5.2.0 web3==7.11.0 python-dotenv==1.0.1 requests==2.32.3

import os, json, time, math, requests
from dotenv import load_dotenv
from eth_abi import encode
from eth_account import Account
from eth_account.messages import encode_defunct
from web3 import Web3

# ========= Config =========
API_HOST = "https://fapi.asterdex.com"
SYMBOL   = "BTCUSDT"  # 疎通確認はメジャー銘柄推奨。SANDUSDT等に後で変更OK
RECV_WINDOW_MS = 50000

# ========= Load keys =========
load_dotenv("keys.env")
USER_RAW   = os.getenv("ASTER_USER")          # 0x....
SIGNER_RAW = os.getenv("ASTER_SIGNER")        # 0x....（通常はUSERと同一）
PRIVATE_KEY= os.getenv("ASTER_PRIVATE_KEY")   # 0x...

assert USER_RAW and SIGNER_RAW and PRIVATE_KEY, "keys.env に ASTER_USER/ASTER_SIGNER/ASTER_PRIVATE_KEY を設定してください"

# 署名・送信時は lower に正規化
USER   = USER_RAW.lower()
SIGNER = SIGNER_RAW.lower()

def _stringify_params(d: dict) -> dict:
    """
    署名対象のparamsを「全て文字列」に変換する（dict/listはJSON文字列に）
    """
    out = {}
    for k, v in d.items():
        if isinstance(v, dict):
            out[k] = json.dumps(_stringify_params(v))
        elif isinstance(v, list):
            tmp = []
            for item in v:
                if isinstance(item, dict):
                    tmp.append(json.dumps(_stringify_params(item)))
                else:
                    tmp.append(str(item))
            out[k] = json.dumps(tmp)
        else:
            out[k] = str(v)
    return out

def _signature_message_hex(sorted_json_str: str, user: str, signer: str, nonce: int) -> str:
    """
    keccak( abi.encode( string(json_str_sorted), address(user), address(signer), uint256(nonce) ) )
    を hex 文字列（0x...）で返す
    """
    payload = encode(["string", "address", "address", "uint256"], [sorted_json_str, user, signer, nonce])
    return Web3.keccak(payload).hex()

def _sign_evm(json_params: dict, user: str, signer: str, nonce: int, private_key: str) -> str:
    """
    上のメッセージをEVM署名（personal_sign風）して signature hex を返す
    """
    msg_hex = _signature_message_hex(
        json.dumps(json_params, sort_keys=True, separators=(",", ":")),
        user, signer, nonce
    )
    signable = encode_defunct(hexstr=msg_hex)
    signed   = Account.sign_message(signable, private_key=private_key)
    sig_hex  = signed.signature.hex()
    return sig_hex if sig_hex.startswith("0x") else "0x" + sig_hex

def post_v3(path: str, params: dict) -> requests.Response:
    """
    /fapi/v3/* を POST (x-www-form-urlencoded, body-only) で叩く
    """
    url = API_HOST + path
    headers = {
        "User-Agent": "PythonApp/1.0",
        "Content-Type": "application/x-www-form-urlencoded",
    }

    # 1) 署名“対象”パラメータ（nonce/user等は含めない）
    base_params = dict(params)  # defensive copy
    base_params["recvWindow"] = RECV_WINDOW_MS
    base_params["timestamp"]  = int(time.time() * 1000)  # ms

    # 2) 署名用に全値を文字列化した dict から、安定化JSON文字列を生成
    to_sign = _stringify_params(base_params)

    # 3) nonce を用意（μs精度）。必要なら ms に変更： int(time.time()*1000)
    nonce = int(time.time() * 1_000_000)

    # 4) 署名
    signature = _sign_evm(to_sign, USER, SIGNER, nonce, PRIVATE_KEY)

    # 5) 送信“本体”に nonce/user/signer/signature を追加
    #    注意：nonce/timestamp/recvWindow は **整数のまま**送る
    send_params = dict(base_params)
    send_params["nonce"]     = nonce          # int
    send_params["user"]      = USER           # lower
    send_params["signer"]    = SIGNER         # lower
    send_params["signature"] = signature      # 0x...

    # 6) POST body only（クエリ文字列や params= は付けない）
    res = requests.post(url, data=send_params, headers=headers, timeout=30)

    # デバッグ（必要なら有効化）
    print("== REQUEST ==")
    try:
        print("URL :", res.request.url)
        print("BODY:", res.request.body)
    except Exception:
        pass
    print("== RESPONSE ==")
    print(res.status_code, res.text[:1000])
    return res

def get_position_risk(symbol: str) -> dict:
    """
    /fapi/v3/positionRisk を叩いて結果を返す（dict）。エラー時は例外。
    """
    res = post_v3("/fapi/v3/positionRisk", {"symbol": symbol})
    if res.status_code != 200:
        raise RuntimeError(f"HTTP {res.status_code}: {res.text}")
    try:
        return res.json()
    except Exception:
        raise RuntimeError(f"Invalid JSON: {res.text}")

if __name__ == "__main__":
    # 疎通チェック
    data = get_position_risk(SYMBOL)
    print("\nParsed JSON:")
    print(json.dumps(data, ensure_ascii=False, indent=2))


== REQUEST ==
URL : https://fapi.asterdex.com/fapi/v3/positionRisk
BODY: symbol=BTCUSDT&recvWindow=50000&timestamp=1758663948105&nonce=1758663948105488&user=0xca44395071da0d5d8177396dfd1de6deb4cb3e71&signer=0xca44395071da0d5d8177396dfd1de6deb4cb3e71&signature=0x490c56404831053faa49156635e699ce5d644b77a3c39373d1fc475c01280c530da094ed7e2d27c01b3ae28043ad0535a9b2db9afd1629bc0066bc535550859d1c
== RESPONSE ==
405 


RuntimeError: HTTP 405: 

In [34]:
# Python 3.9.6
# pip install eth-account==0.13.7 eth-abi==5.2.0 web3==7.11.0 python-dotenv==1.0.1 requests==2.32.3

import os, json, time, requests
from dotenv import load_dotenv
from eth_abi import encode
from eth_account import Account
from eth_account.messages import encode_defunct
from web3 import Web3

API_HOST = "https://fapi.asterdex.com"
ENDPOINT = "/fapi/v3/positionRisk"
SYMBOL   = "BTCUSDT"      # 疎通確認はメジャー銘柄で

RECV_WINDOW_MS = 50000

load_dotenv("keys.env")
USER_RAW   = os.getenv("ASTER_USER")
SIGNER_RAW = os.getenv("ASTER_SIGNER")
PRIVATE_KEY= os.getenv("ASTER_PRIVATE_KEY")
assert USER_RAW and SIGNER_RAW and PRIVATE_KEY, "keys.env に ASTER_USER/ASTER_SIGNER/ASTER_PRIVATE_KEY を設定してください"

# 送信はlowercaseで
USER   = USER_RAW.lower()
SIGNER = SIGNER_RAW.lower()

def _stringify_params(d: dict) -> dict:
    """署名対象paramsを全て文字列化（list/dictはJSON文字列化）"""
    out = {}
    for k, v in d.items():
        if isinstance(v, dict):
            out[k] = json.dumps(_stringify_params(v))
        elif isinstance(v, list):
            out[k] = json.dumps([json.dumps(_stringify_params(i)) if isinstance(i, dict) else str(i) for i in v])
        else:
            out[k] = str(v)
    return out

def _msg_hex(sorted_json_str: str, user: str, signer: str, nonce: int) -> str:
    payload = encode(["string", "address", "address", "uint256"], [sorted_json_str, user, signer, nonce])
    return Web3.keccak(payload).hex()

def _sign_evm(json_params_str_sorted: str, user: str, signer: str, nonce: int, private_key: str) -> str:
    msg_hex = _msg_hex(json_params_str_sorted, user, signer, nonce)
    signable = encode_defunct(hexstr=msg_hex)
    sig = Account.sign_message(signable, private_key=private_key).signature.hex()
    return sig if sig.startswith("0x") else "0x" + sig

def get_position_risk(symbol: str) -> dict:
    url = API_HOST + ENDPOINT
    headers = {"User-Agent": "PythonApp/1.0"}

    # 署名“対象”の基本パラメータ（nonce/user/signer/signatureは含めない）
    base_params = {
        "symbol": symbol,
        "recvWindow": RECV_WINDOW_MS,           # intのまま
        "timestamp": int(time.time() * 1000),   # ms, intのまま
    }

    # 署名用に全値を文字列化 → ソートして安定化
    to_sign = _stringify_params(base_params)
    sorted_str = json.dumps(to_sign, sort_keys=True, separators=(",", ":"))

    # nonce はμs単位（必要ならmsに切替可）
    nonce = int(time.time() * 1_000_000)

    # 署名
    signature = _sign_evm(sorted_str, USER, SIGNER, nonce, PRIVATE_KEY)

    # 実送信パラメータ（GETはクエリに載せる。bodyは使わない）
    send_params = dict(base_params)
    send_params.update({
        "nonce": nonce,          # intのまま
        "user": USER,            # lower
        "signer": SIGNER,        # lower
        "signature": signature,  # 0x...
    })

    # GET (query only)
    res = requests.get(url, params=send_params, headers=headers, timeout=30)

    # デバッグ出力
    print("== REQUEST ==")
    try:
        print("URL :", res.request.url)
        print("BODY:", res.request.body)  # GETなので通常 None
    except Exception:
        pass
    print("== RESPONSE ==")
    print(res.status_code, res.text[:1000])

    if res.status_code != 200:
        raise RuntimeError(f"HTTP {res.status_code}: {res.text}")

    try:
        return res.json()
    except Exception:
        raise RuntimeError(f"Invalid JSON: {res.text}")

if __name__ == "__main__":
    data = get_position_risk(SYMBOL)
    print("\nParsed JSON:")
    print(json.dumps(data, ensure_ascii=False, indent=2))


== REQUEST ==
URL : https://fapi.asterdex.com/fapi/v3/positionRisk?symbol=BTCUSDT&recvWindow=50000&timestamp=1758664051219&nonce=1758664051219179&user=0xca44395071da0d5d8177396dfd1de6deb4cb3e71&signer=0xca44395071da0d5d8177396dfd1de6deb4cb3e71&signature=0x815dfa6e03ed9feb5c8bfdb9a52cb62379755ebe8e75dfdcd688d459cdc852cd7d69978ac4aadb0a89d61573819b20c9b15f37823b3e4690f6268cbafca24d481b
BODY: None
== RESPONSE ==
500 <!DOCTYPE html>
<html>
  <head>
    <style>
      * {
        box-sizing: border-box;
      }
      html,
      body {
        margin: 0;
        border: 0;
      }
      .p-8 {
        padding: 8px;
      }
      .bg-0b0d12 {
        background-color: #0b0d12;
      }
      .minH-100p {
        min-height: 100%;
      }
      .absolute {
        position: absolute;
      }
      .w-100p {
        width: 100%;
      }
      .bottom-0 {
        bottom: 0px;
      }
      .top-0 {
        top: 0px;
      }
      .column-toCenterX {
        align-items: center;
      }
      .fle

RuntimeError: HTTP 500: <!DOCTYPE html>
<html>
  <head>
    <style>
      * {
        box-sizing: border-box;
      }
      html,
      body {
        margin: 0;
        border: 0;
      }
      .p-8 {
        padding: 8px;
      }
      .bg-0b0d12 {
        background-color: #0b0d12;
      }
      .minH-100p {
        min-height: 100%;
      }
      .absolute {
        position: absolute;
      }
      .w-100p {
        width: 100%;
      }
      .bottom-0 {
        bottom: 0px;
      }
      .top-0 {
        top: 0px;
      }
      .column-toCenterX {
        align-items: center;
      }
      .flexDirection-column {
        flex-direction: column;
      }
      .flex {
        display: flex;
      }
      .top-50p {
        top: 50%;
      }
      .left-50p {
        left: 50%;
      }
      .transform-translateX--50p-translateY--70p {
        transform: translateX(-50%) translateY(-70%);
      }
      .w-180 {
        width: 180px;
      }
      @media (min-width: 640px) {
        

In [36]:
# Python 3.9.6
# pip install requests==2.32.3 python-dotenv==1.0.1

import os, time, hmac, hashlib, requests, urllib.parse
from dotenv import load_dotenv

API_HOST = "https://fapi.asterdex.com"
API_KEY_HEADER = "X-MBX-APIKEY"   # Binance互換

load_dotenv("keys.env")
API_KEY    = os.getenv("ASTER_API_KEY")
API_SECRET = os.getenv("ASTER_API_SECRET")

assert API_KEY and API_SECRET, "keys.env に ASTER_API_KEY / ASTER_API_SECRET を設定してください"

def _sign(query_str: str, secret: str) -> str:
    return hmac.new(secret.encode("utf-8"), query_str.encode("utf-8"), hashlib.sha256).hexdigest()

def _get(path: str, params: dict):
    url = API_HOST + path
    # 1) timestamp/recvWindow を付与（intのまま）
    params = dict(params)
    params["timestamp"]  = int(time.time() * 1000)   # ms
    params["recvWindow"] = 50000

    # 2) クエリ文字列を生成
    qs = urllib.parse.urlencode(params, doseq=True)

    # 3) HMAC署名を付与
    signature = _sign(qs, API_SECRET)

    # 4) GET（署名はクエリ末尾）
    full_url = f"{url}?{qs}&signature={signature}"
    headers = {
        API_KEY_HEADER: API_KEY,
        "User-Agent": "PythonApp/1.0",
    }
    r = requests.get(full_url, headers=headers, timeout=30)
    print("== REQUEST ==")
    print(full_url)
    print("== RESPONSE ==")
    print(r.status_code, r.text[:1000])
    r.raise_for_status()
    return r.json()

if __name__ == "__main__":
    # symbol 省略可の実装もありますが、まずは付けて安全に
    data = _get("/fapi/v2/positionRisk", {"symbol": "BTCUSDT"})
    print("\nParsed JSON:")
    print(data)


== REQUEST ==
https://fapi.asterdex.com/fapi/v2/positionRisk?symbol=BTCUSDT&timestamp=1758664165196&recvWindow=50000&signature=e6aa7b3c5117cccce83a8cbde64c26de89ccfdd8fe2d9062ac3572017a945e1c
== RESPONSE ==
200 [{"symbol":"BTCUSDT","positionAmt":"-0.019","entryPrice":"112100.1","markPrice":"112067.06130435","unRealizedProfit":"0.62773521","liquidationPrice":"117262.32679393","leverage":"20","maxNotionalValue":"12500000","marginType":"cross","isolatedMargin":"0.00000000","isAutoAddMargin":"false","positionSide":"BOTH","notional":"-2129.27416478","isolatedWallet":"0","updateTime":1758663471595}]

Parsed JSON:
[{'symbol': 'BTCUSDT', 'positionAmt': '-0.019', 'entryPrice': '112100.1', 'markPrice': '112067.06130435', 'unRealizedProfit': '0.62773521', 'liquidationPrice': '117262.32679393', 'leverage': '20', 'maxNotionalValue': '12500000', 'marginType': 'cross', 'isolatedMargin': '0.00000000', 'isAutoAddMargin': 'false', 'positionSide': 'BOTH', 'notional': '-2129.27416478', 'isolatedWallet': '

In [48]:
# -*- coding: utf-8 -*-
# Python 3.9.6
# pip install requests==2.32.3 python-dotenv==1.0.1 pybit==5.7.0

import os, time, hmac, hashlib, urllib.parse, requests, math, threading
from dataclasses import dataclass
from typing import Optional, Tuple
from dotenv import load_dotenv
from pybit.unified_trading import HTTP as BybitHTTP
from decimal import Decimal, ROUND_FLOOR, ROUND_CEILING

# ================== 設定 ==================
SYMBOL         = "BTCUSDT"
NOTIONAL_USD   = 1000          # 片側の想定USDサイズ（両所で同サイズ）
BTC_QTY_STEP   = 0.001          # 取引所の最小数量刻み（BTCは 0.001 が無難）
POLL_SEC       = 120            # 2分ごとチェック
REENTER_DELAY  = 60             # 1分後に再エントリー
CYCLE_RESET_S  = 60 * 60        # 1.5時間で一旦リセット
SL_PCT         = 0.02           # ±3% ストップレンジ
RECV_WINDOW    = 50000
USER_AGENT     = "FundingArbBot/1.0"

ASTER_HOST     = "https://fapi.asterdex.com"  # v2 HMAC
ASTER_KEY_HDR  = "X-MBX-APIKEY"

# Bybit endpoints are handled by pybit.

USE_MANUAL_PRECISION = True

ASTER_PRECISION = {
    "price_tick": Decimal("0.1"),
    "price_precision": 1,
    "qty_step": Decimal("0.001"),
    "qty_min": Decimal("0.001"),
    "qty_precision": 3,
}

BYBIT_PRECISION = {
    "price_tick": Decimal("0.1"),
    "price_precision": 2,
    "qty_step": Decimal("0.001"),
    "qty_min": Decimal("0.001"),
    "qty_precision": 3,
}


# ================== 共通ユーティリティ ==================
def _now_ms() -> int:
    return int(time.time() * 1000)

def _hmac_sha256(secret: str, msg: str) -> str:
    return hmac.new(secret.encode("utf-8"), msg.encode("utf-8"), hashlib.sha256).hexdigest()

def _round_to_step(val: Decimal, step: Decimal, direction: str) -> Decimal:
    units = (val / step).to_integral_value(rounding=(ROUND_CEILING if direction == "up" else ROUND_FLOOR))
    return units * step

def _limit_decimals(val: Decimal, decimals: int) -> Decimal:
    if decimals <= 0:
        return Decimal(int(val))
    q = Decimal(1).scaleb(-decimals)  # 10^-decimals
    # 超過桁は切り捨て（上方向が必要なら呼び出し側で step 丸めを up にする）
    return val.quantize(q, rounding=ROUND_FLOOR)

def _round_to_step_dec(val: Decimal, step: Decimal, direction: str) -> Decimal:
    units = (val / step).to_integral_value(rounding=(ROUND_CEILING if direction == "up" else ROUND_FLOOR))
    return units * step

def _limit_decimals_dec(val: Decimal, decimals: int) -> Decimal:
    if decimals <= 0: return Decimal(int(val))
    q = Decimal(1).scaleb(-decimals)
    return val.quantize(q, rounding=ROUND_FLOOR)

def round_down_qty(qty: float, step: float) -> float:
    q = Decimal(str(qty))
    s = Decimal(str(step))
    units = (q / s).to_integral_value(rounding=ROUND_FLOOR)
    return float(units * s)



# ================== Aster v2 HMAC クライアント ==================
@dataclass
class AsterClient:
    api_key: str
    api_secret: str
    host: str = ASTER_HOST
    # manual precision（必須）
    price_tick: Decimal = Decimal("0.1")
    price_precision: int = 1
    qty_step: Decimal = Decimal("0.001")
    qty_min: Decimal = Decimal("0.001")
    qty_precision: int = 3

    # ---- HTTP ----
    def _request(self, method: str, path: str, params: dict):
        url = self.host + path
        q = dict(params or {})
        q.setdefault("recvWindow", RECV_WINDOW)
        q["timestamp"] = _now_ms()
        qs = urllib.parse.urlencode(q, doseq=True)
        sig = _hmac_sha256(self.api_secret, qs)
        headers = {ASTER_KEY_HDR: self.api_key, "User-Agent": USER_AGENT}

        if method.upper() == "GET":
            full_url = f"{url}?{qs}&signature={sig}"
            r = requests.get(full_url, headers=headers, timeout=30)
        elif method.upper() == "POST":
            headers["Content-Type"] = "application/x-www-form-urlencoded"
            body = f"{qs}&signature={sig}"
            r = requests.post(url, data=body, headers=headers, timeout=30)
        elif method.upper() == "DELETE":
            headers["Content-Type"] = "application/x-www-form-urlencoded"
            body = f"{qs}&signature={sig}"
            r = requests.delete(url, data=body, headers=headers, timeout=30)
        else:
            raise ValueError("Unsupported method")

        if r.status_code >= 400:
            raise RuntimeError(f"Aster {method} {path} failed: {r.status_code} {r.text[:500]}")
        try:
            return r.json()
        except Exception:
            return r.text

    # ---- precision helpers ----
    def fmt_qty(self, qty: float, direction: str = "down") -> str:
        v = Decimal(str(qty))
        v = _round_to_step(v, self.qty_step, direction)
        if v < max(self.qty_min, self.qty_step):
            v = self.qty_step
        v = _limit_decimals(v, self.qty_precision)
        return f"{v:.{self.qty_precision}f}".rstrip("0").rstrip(".")

    def fmt_price(self, price: float, direction: str) -> str:
        v = Decimal(str(price))
        v = _round_to_step(v, self.price_tick, direction)
        v = _limit_decimals(v, self.price_precision)
        return f"{v:.{self.price_precision}f}".rstrip("0").rstrip(".")

    # ---- public APIs ----
    def get_funding_rate(self, symbol: str) -> float:
        d = self._request("GET", "/fapi/v1/premiumIndex", {"symbol": symbol})
        item = d[0] if isinstance(d, list) else d
        return float(item.get("lastFundingRate") or item.get("fundingRate") or 0.0)

    def get_mark_price(self, symbol: str) -> float:
        d = self._request("GET", "/fapi/v1/premiumIndex", {"symbol": symbol})
        item = d[0] if isinstance(d, list) else d
        return float(item["markPrice"])

    def get_position(self, symbol: str):
        d = self._request("GET", "/fapi/v2/positionRisk", {"symbol": symbol})
        item = d[0] if isinstance(d, list) else d
        qty = float(item.get("positionAmt", 0.0))   # +long / -short
        ep  = float(item.get("entryPrice", 0.0))
        return qty, ep

    def place_market(self, symbol: str, side: str, qty: float, reduce_only: bool = False):
        q_str = self.fmt_qty(qty, direction="down")
        p = {
            "symbol": symbol,
            "side": side,  # "BUY"/"SELL"
            "type": "MARKET",
            "quantity": q_str,
            "positionSide": "BOTH",
        }
        if reduce_only:
            p["reduceOnly"] = "true"
        return self._request("POST", "/fapi/v1/order", p)

    def place_stop_market_close(self, symbol: str, trigger_price: float, side_to_close: str, fallback_qty: float = None):
        # SELL=ロング解消→下方向丸め, BUY=ショート解消→上方向丸め
        direction = "down" if side_to_close == "SELL" else "up"
        stop_str  = self.fmt_price(trigger_price, direction=direction)

        # A: closePosition=true（reduceOnlyは一緒に送らない）
        pA = {
            "symbol": symbol, "side": side_to_close, "type": "STOP_MARKET",
            "stopPrice": stop_str, "closePosition": "true", "positionSide": "BOTH",
        }
        try:
            return self._request("POST", "/fapi/v1/order", pA)
        except RuntimeError as e:
            print("[Aster STOP closePosition] failed, fallback to qty+reduceOnly:", e)

        # B: quantity + reduceOnly=true
        if not fallback_qty or fallback_qty <= 0:
            try:
                pos = self._request("GET", "/fapi/v2/positionRisk", {"symbol": symbol})
                item = pos[0] if isinstance(pos, list) else pos
                fallback_qty = abs(float(item.get("positionAmt", 0.0)))
            except Exception:
                fallback_qty = 0.0
        if fallback_qty <= 0:
            raise RuntimeError("Fallback stop failed and no open position qty found.")

        q_str = self.fmt_qty(fallback_qty, direction="down")
        pB = {
            "symbol": symbol, "side": side_to_close, "type": "STOP_MARKET",
            "stopPrice": stop_str, "quantity": q_str, "reduceOnly": "true", "positionSide": "BOTH",
        }
        return self._request("POST", "/fapi/v1/order", pB)

    def cancel_all(self, symbol: str):
        try:
            orders = self._request("GET", "/fapi/v1/openOrders", {"symbol": symbol})
            if isinstance(orders, list):
                for o in orders:
                    oid = o.get("orderId")
                    if oid is not None:
                        self._request("DELETE", "/fapi/v1/order", {"symbol": symbol, "orderId": oid})
        except Exception as e:
            print("[Aster] cancel_all error:", e)

# ================== Bybit クライアント（pybit） ==================
@dataclass
class BybitClient:
    api_key: str
    api_secret: str
    testnet: bool = False
    # manual precision（必須）
    price_tick: Decimal = Decimal("0.5")
    price_precision: int = 1
    qty_step: Decimal = Decimal("0.001")
    qty_min: Decimal = Decimal("0.001")
    qty_precision: int = 3

    def __post_init__(self):
        self.session = BybitHTTP(testnet=self.testnet, api_key=self.api_key, api_secret=self.api_secret)

    # precision helpers
    def fmt_qty(self, qty: float, direction: str = "down") -> str:
        v = Decimal(str(qty))
        v = _round_to_step_dec(v, self.qty_step, direction)
        if v < max(self.qty_min, self.qty_step):
            v = self.qty_step
        v = _limit_decimals_dec(v, self.qty_precision)
        return f"{v:.{self.qty_precision}f}".rstrip("0").rstrip(".")

    def fmt_price(self, price: float, direction: str) -> str:
        v = Decimal(str(price))
        v = _round_to_step_dec(v, self.price_tick, direction)
        v = _limit_decimals_dec(v, self.price_precision)
        return f"{v:.{self.price_precision}f}".rstrip("0").rstrip(".")

    # public APIs
    def get_funding_rate(self, symbol: str) -> float:
        r = self.session.get_tickers(category="linear", symbol=symbol)
        return float(r["result"]["list"][0].get("fundingRate", 0.0))

    def get_mark_price(self, symbol: str) -> float:
        r = self.session.get_tickers(category="linear", symbol=symbol)
        return float(r["result"]["list"][0]["markPrice"])

    def get_position(self, symbol: str):
        r = self.session.get_positions(category="linear", symbol=symbol)
        lst = r["result"]["list"]
        if not lst:
            return 0.0, 0.0
        pos = lst[0]
        size = float(pos.get("size", 0.0))
        side = pos.get("side", "Buy")
        qty = size if side == "Buy" else -size
        ep  = float(pos.get("avgPrice", 0.0))
        return qty, ep

    def place_market(self, symbol: str, side: str, qty: float, reduce_only: bool = False):
        q_str = self.fmt_qty(qty)
        return self.session.place_order(
            category="linear",
            symbol=symbol,
            side=side.title(),       # "Buy"/"Sell"
            orderType="Market",
            qty=q_str,
            reduceOnly=reduce_only
        )

    def place_stop_market_close(self, symbol: str, trigger_price: float, side_to_close: str, qty: float):
        # SELL=long close -> 下方向, BUY=short close -> 上方向
        direction = "down" if side_to_close == "SELL" else "up"
        trig = self.fmt_price(trigger_price, direction)
        q_str = self.fmt_qty(qty)               # ★ StopOrder にも数量を明示
        trigger_dir = 2 if side_to_close == "SELL" else 1

        # 注意: qty を送る場合は closePosition は送らない（競合するため）
        return self.session.place_order(
            category="linear",
            symbol=symbol,
            side=side_to_close.title(),         # "Buy"/"Sell"
            orderType="Market",
            qty=q_str,                           # ★必須
            reduceOnly=True,
            closeOnTrigger=True,
            tpslMode="Full",
            triggerPrice=trig,
            triggerDirection=trigger_dir,
            timeInForce="GoodTillCancel",
            orderFilter="StopOrder",
            triggerBy="MarkPrice",
            positionIdx=0,                       # 片面/BOTH
        )
    
    def cancel_all(self, symbol: str):
        try:
            self.session.cancel_all_orders(category="linear", symbol=symbol)
        except Exception as e:
            print("[Bybit] cancel_all error:", e)

# ================== 戦略ロジック ==================
@dataclass
class Leg:
    ex_name: str                  # "ASTER" / "BYBIT"
    side: str                     # "LONG"/"SHORT"
    qty: float
    entry_price: float
    sl_price: float
    opened_ms: int

class FundingArbBot:
    def __init__(self, aster: AsterClient, bybit: BybitClient):
        self.aster = aster
        self.bybit = bybit
        self.legs: Tuple[Optional[Leg], Optional[Leg]] = (None, None)

    def _decide_sides(self, fr_aster: float, fr_bybit: float) -> Tuple[str, str]:
        # 高い方をショート、低い方をロング
        if fr_aster > fr_bybit:
            return ("SHORT", "LONG")   # Aster, Bybit
        elif fr_aster < fr_bybit:
            return ("LONG", "SHORT")
        else:
            # 同値なら何もしない or どちらか固定。ここでは固定で Aster=SHORT, Bybit=LONG
            return ("SHORT", "LONG")

    def _compute_qty(self, price: float) -> float:
        raw = NOTIONAL_USD / price
        return max(round_down_qty(raw, BTC_QTY_STEP), BTC_QTY_STEP)

    def _place_leg(self, ex: str, side: str, qty: float, price_now: float) -> Leg:
        # どの取引所か
        if ex == "ASTER":
            cli = self.aster
        else:
            cli = self.bybit

        if side == "LONG":
            # 成行でロング
            cli.place_market(SYMBOL, "BUY", qty, reduce_only=False)
            sl_price = price_now * (1 - SL_PCT)
            # SLはロング解消=SELL
            try:
                if ex == "ASTER":
                    self.aster.place_stop_market_close(SYMBOL, sl_price, side_to_close="SELL", fallback_qty=qty)
                else:
                    self.bybit.place_stop_market_close(SYMBOL, sl_price, side_to_close="SELL", qty=qty)
            except Exception as e:
                print(f"[{ex} STOP(LONG close) setup failed]", e)
        else:
            # 成行でショート
            cli.place_market(SYMBOL, "SELL", qty, reduce_only=False)
            sl_price = price_now * (1 + SL_PCT)
            # SLはショート解消=BUY
            try:
                if ex == "ASTER":
                    self.aster.place_stop_market_close(SYMBOL, sl_price, side_to_close="BUY", fallback_qty=qty)
                else:
                    self.bybit.place_stop_market_close(SYMBOL, sl_price, side_to_close="BUY", qty=qty)
            except Exception as e:
                print(f"[{ex} STOP(SHORT close) setup failed]", e)

        # ★どの経路でも必ず返す
        return Leg(ex, side, qty, price_now, sl_price, _now_ms())


    def _get_positions(self) -> Tuple[float, float, float, float]:
        # qty(+long/-short), entryPrice
        a_qty, a_ep = self.aster.get_position(SYMBOL)
        b_qty, b_ep = self.bybit.get_position(SYMBOL)
        return a_qty, a_ep, b_qty, b_ep

    def _is_open(self, qty: float) -> bool:
        return abs(qty) > 1e-9

    def open_basket(self):
        # funding取得
        fr_a = self.aster.get_funding_rate(SYMBOL)
        fr_b = self.bybit.get_funding_rate(SYMBOL)
        print(f"[funding] Aster={fr_a:.6f}, Bybit={fr_b:.6f}")

        # マーク価格
        mp_a = self.aster.get_mark_price(SYMBOL)
        mp_b = self.bybit.get_mark_price(SYMBOL)

        # サイド決定（Aster側, Bybit側）
        side_a, side_b = self._decide_sides(fr_a, fr_b)
        qty_a = self._compute_qty(mp_a)
        qty_b = self._compute_qty(mp_b)

        # エントリー
        leg_a = self._place_leg("ASTER", side_a, qty_a, mp_a)
        leg_b = self._place_leg("BYBIT", side_b, qty_b, mp_b)
        self.legs = (leg_a, leg_b)
        print(f"[enter] Aster {side_a} {qty_a} @~{mp_a:.2f}, SL~{leg_a.sl_price:.2f}")
        print(f"[enter] Bybit {side_b} {qty_b} @~{mp_b:.2f}, SL~{leg_b.sl_price:.2f}")

    def close_all(self, reason: str):
        print(f"[close_all] reason={reason}")
        # reduce-only で反対成行
        try:
            a_qty, _, b_qty, _ = self._get_positions()
            if self._is_open(a_qty):
                side = "SELL" if a_qty > 0 else "BUY"
                self.aster.place_market(SYMBOL, side, abs(a_qty), reduce_only=True)
            self.aster.cancel_all(SYMBOL)
        except Exception as e:
            print("[Aster close] err:", e)

        try:
            a_qty, _, b_qty, _ = self._get_positions()
            if self._is_open(b_qty):
                side = "SELL" if b_qty > 0 else "BUY"
                self.bybit.place_market(SYMBOL, side, abs(b_qty), reduce_only=True)
            self.bybit.cancel_all(SYMBOL)
        except Exception as e:
            print("[Bybit close] err:", e)

        self.legs = (None, None)

    def check_cycle_reset(self) -> bool:
        # 1.5h 経過で True
        l1, l2 = self.legs
        if not l1 or not l2:
            return False
        opened = min(l1.opened_ms, l2.opened_ms)
        return (_now_ms() - opened) >= CYCLE_RESET_S * 1000

    def poll_once(self) -> Optional[str]:
        # 片側が消えていたら "REENTER"、両方OKなら None
        a_qty, _, b_qty, _ = self._get_positions()
        a_open = self._is_open(a_qty)
        b_open = self._is_open(b_qty)

        if a_open and b_open:
            return None
        if (a_open and not b_open) or (b_open and not a_open):
            return "REENTER"
        # 両方消えてたら、そのまま再エントリー合図
        return "REENTER"

    def run(self):
        # 初回エントリー
        self.open_basket()
        last_reset = time.time()

        while True:
            time.sleep(POLL_SEC)

            # 1.5時間でリセット？
            if self.check_cycle_reset():
                self.close_all("cycle_reset_1.5h")
                self.open_basket()
                last_reset = time.time()
                continue

            action = self.poll_once()
            if action == "REENTER":
                print("[reenter] one side closed, syncing hedge...")
                # 両方クローズしてから再エントリー
                self.close_all("hedge_sync")
                time.sleep(REENTER_DELAY)
                self.open_basket()

# ================== 起動 ==================
def main():
    load_dotenv("keys.env")
    aster_kwargs = {}
    bybit_kwargs = {}

    if USE_MANUAL_PRECISION:
        aster_kwargs.update(
            price_tick=ASTER_PRECISION["price_tick"],
            price_precision=ASTER_PRECISION["price_precision"],
            qty_step=ASTER_PRECISION["qty_step"],
            qty_min=ASTER_PRECISION["qty_min"],
            qty_precision=ASTER_PRECISION["qty_precision"],
        )
        bybit_kwargs.update(
            price_tick=BYBIT_PRECISION["price_tick"],
            price_precision=BYBIT_PRECISION["price_precision"],
            qty_step=BYBIT_PRECISION["qty_step"],
            qty_min=BYBIT_PRECISION["qty_min"],
            qty_precision=BYBIT_PRECISION["qty_precision"],
        )

    aster = AsterClient(
        api_key=os.getenv("ASTER_API_KEY") or "",
        api_secret=os.getenv("ASTER_API_SECRET") or "",
        **aster_kwargs
    )
    bybit = BybitClient(
        api_key=os.getenv("BYBIT_API_KEY") or "",
        api_secret=os.getenv("BYBIT_API_SECRET") or "",
        testnet=(os.getenv("BYBIT_TESTNET", "false").lower() == "true"),
        **bybit_kwargs
    )
    bot = FundingArbBot(aster, bybit)
    bot.run()

if __name__ == "__main__":
    main()


[funding] Aster=0.000100, Bybit=0.000000
[enter] Aster SHORT 0.008 @~112153.04, SL~114396.10
[enter] Bybit LONG 0.008 @~112132.75, SL~109890.10


KeyboardInterrupt: 